In [4]:
import networkx as nx
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
from itertools import product
from sklearn.model_selection import train_test_split
from copy import deepcopy

In [39]:
# Generate fake edges
N_fake = 10000
edges_fake = []
while len(edges_fake) < N_fake:
    u = np.random.choice(subG.nodes)
    v = np.random.choice(subG.nodes)
    if u != v and not subG.has_edge(u,v) and not (u,v) in edges_hidden and not (u,v) in edges_fake:
        edges_fake.append((u,v))

In [ ]:
# Generate fake edges
N_fake = 1000
edges_fake = []
while len(edges_fake) < N_fake:
    u = np.random.choice(subG.nodes)
    v = np.random.choice(subG.nodes)
    if u != v and not subG.has_edge(u,v) and not (u,v) in edges_hidden and not (u,v) in edges_fake:
        edges_fake.append((u,v))

In [43]:
test = io.load('test_public')
test.columns = map(str.lower, test.columns)
test.set_index('id', inplace = True)
test = test.applymap(str)
test_pairs = [(source, sink) for i, source, sink in test.itertuples()]; test_pairs[:10]

2019-06-21 21:15:50,598 - kedro.io.data_catalog - INFO - Loading data from `test_public` (CSVLocalDataSet)...


[('2184483', '1300190'),
 ('3151356', '1452193'),
 ('1579396', '193159'),
 ('1406432', '2481036'),
 ('2389638', '593017'),
 ('228206', '212805'),
 ('1237964', '879115'),
 ('3318124', '1840575'),
 ('4522929', '1552625'),
 ('3406737', '3781412')]

In [53]:
print('Number of fake edges:', len(edges_fake))
print('Overlap with test data:', len([x for x in edges_fake if x in test_pairs]))

Number of fake edges: 1000
Overlap with test data: 0


In [56]:
# Create labels for edges and randomly shuffle
labels = [(e, 1) for e in edges_hidden[:1000]] + [(e, 0) for e in edges_fake]
np.random.shuffle(labels)
labels = pd.DataFrame(labels, columns = ['edge', 'valid']); labels

,edge,valid
0,"(4170678, 277031)",0
1,"(4810797, 2776422)",1
2,"(20388, 3733604)",1
3,"(689629, 1859585)",1
4,"(2626939, 4312872)",0
5,"(215319, 4523947)",0
6,"(4043944, 1840755)",0
7,"(1988024, 4651821)",0
8,"(2547985, 4562280)",0
9,"(868308, 401392)",1


In [57]:
from networkx.algorithms import link_prediction as lp
def similarity(graph, edges):
    return pd.DataFrame({
        'ra': [x for u, v, x in lp.resource_allocation_index(graph, edges)],
        'jc': [x for u, v, x in lp.jaccard_coefficient(graph, edges)],
        'aa': [x for u, v, x in lp.adamic_adar_index(graph, edges)],
        'pa': [x for u, v, x in lp.preferential_attachment(graph, edges)]
    })

In [59]:
sim_train = similarity(G, labels.edge)
sim_train.head(n = 10)

,ra,jc,aa,pa
0,0.000000,0.000000,0.000000,1
1,0.000117,0.000261,0.633759,401430
2,0.000329,0.000001,0.124694,1525314
3,0.007505,0.000510,1.336801,235416
4,0.000000,0.000000,0.000000,6
5,0.000000,0.000000,0.000000,2
6,0.000000,0.000000,0.000000,1
7,0.000000,0.000000,0.000000,3
8,0.000004,0.333333,0.079918,3
9,0.000095,0.000182,0.286737,131536


In [ ]:
sim_test = similarity(sub_G_test, labels_test.edge)
sim_test.head(n = 10)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create training and test data
N_classes = sim_train.shape[1]
X_train = sim_train.values
X_test = sim_test.values
y_train = labels_train.valid.values
y_test = labels_test.valid.values

# Re-scale training data
non_stand_X_train = X_train
non_stand_X_test = X_test
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print("Mean for training features (before standardisation):", non_stand_X_train.mean(axis=0))
print("Mean for training features (after standardisation):", X_train.mean(axis =  0))
print("Mean for test features (after standardisation):", X_test.mean(axis =  0))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def evaluation(y_test, y_pred):
    # Model Accuracy
    print("Accuracy:", accuracy_score(y_test, y_pred))
    # Model Precision
    print("Precision:", precision_score(y_test, y_pred))
    # Model Recall
    print("Recall:", recall_score(y_test, y_pred))
    # Model f1_score
    print("f1_score:", f1_score(y_test, y_pred))
    # AUC
    print("AUC:", roc_auc_score(y_test, y_pred))

In [ ]:
# Naive bayes
from sklearn.naive_bayes import BernoulliNB

clf_NB = BernoulliNB()
clf_NB.fit(X_train, y_train)
NB_pred = clf_NB.predict(X_test)
evaluation(y_test, NB_pred)

In [ ]:
# Logistic regression
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression(solver = 'liblinear')
clf_lr.fit(X_train, y_train)
lr_pred = clf_logit.predict(X_test)
evaluation(y_test, lr_pred)

In [ ]:
# Decision tree model
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

clf_tree = DecisionTreeClassifier()
clf_tree = clf_tree.fit(X_train, y_train)
tree_pred = clf_tree.predict(X_test)
evaluation(y_test, tree_pred)

In [ ]:
# Support vector machine
from sklearn.svm import SVC

clf_svm = SVC(kernel = 'rbf', gamma = 'scale', C = 10, probability = True)
clf_svm.fit(X_train, y_train)
svm_pred = clf_svm.predict(X_test)
evaluation(y_test, svm_pred)

In [ ]:
# Random forest
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(n_estimators=500, random_state=0)
clf_rf.fit(X_train, y_train)
rf_pred = clf_forest.predict(X_test)
evaluation(y_test, rf_pred)

In [ ]:
# Ada boost
from sklearn.ensemble import AdaBoostClassifier

clf_ada = AdaBoostClassifier(n_estimators = 500)
clf_ada.fit(X_train, y_train)
ada_pred = clf_ada.predict(X_test)
evaluation(y_test, ada_pred)

In [ ]:
# Neural network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

N_features = X_train.shape[1]
BATCH_SIZE = 128

clf_nn = Sequential()
clf_nn.add(Dense(N_features, activation='tanh', use_bias = True))
clf_nn.add(Dense(N_features, activation='tanh', use_bias = True))
clf_nn.add(Dense(N_features, activation='tanh', use_bias = True))
clf_nn.add(Dense(N_features, activation='tanh', use_bias = True))
clf_nn.add(Dense(N_features, activation='tanh', use_bias = True))
clf_nn.add(Dense(1, activation='sigmoid', use_bias=True))

clf_nn.compile(optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fit the clf_nn
clf_nn.fit(X_train, y_train, epochs=20, batch_size=BATCH_SIZE)

In [ ]:
nn_pred = [x[0] for x in clf_nn.predict_classes(X_test, batch_size = BATCH_SIZE)]
evaluation(y_test, nn_pred)

In [ ]:
from  sklearn.ensemble import VotingClassifier
clf_vote = VotingClassifier(
    estimators  = [('lr', clf_logit),
                   ('rf', clf_forest),
                   ('svm', clf_svm),
                   ('ada', clf_ada)],
    voting = "soft"
)
clf_vote.fit(X_train, y_train)
vote_pred = clf_vote.predict(X_test)
evaluation(y_test, vote_pred)

In [ ]:
y_prob_test = pd.DataFrame(
    {'lr': [x[1] for x in clf_lr.predict_proba(X_test)],
     'rf': [x[1] for x in clf_rf.predict_proba(X_test)],
     'svm': [x[1] for x in clf_svm.predict_proba(X_test)],
     'ada': [x[1] for x in clf_ada.predict_proba(X_test)],
     'nn': [x[0] for x in clf_nn.predict(X_test)]
})
y_prob_test.head(n = 10)

In [ ]:
y_pred = pd.DataFrame(
    {'lr': lr_pred,
     'rf': rf_pred,
     'svm': svm_pred,
     'ada': ada_pred,
     'nn': nn_pred
})
y_pred.head()

In [ ]:
def majority_vote(votes):
    return int(sum(votes) > 0.5 * len(votes))

y_maj = y_pred.apply(majority_vote, axis = 1)
evaluation(y_test, y_maj)

In [ ]:
y_any = y_pred.apply(np.mean, axis = 1).apply(int)
evaluation(y_test, y_any)

In [ ]:
y_prob_train = pd.DataFrame(
    {'lr': [x[1] for x in clf_lr.predict_proba(X_train)],
     'rf': [x[1] for x in clf_rf.predict_proba(X_train)],
     'svm': [x[1] for x in clf_svm.predict_proba(X_train)],
     'ada': [x[1] for x in clf_ada.predict_proba(X_train)],
     'nn': [x[0] for x in clf_nn.predict(X_train)]
})
y_prob_train.head(n = 10)

In [ ]:
y_mean = y_prob_test.apply(np.mean, axis = 1); y_mean.head(n = 10)

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt 
fpr, tpr, thresholds = roc_curve(y_test, y_mean)
youden = tpr - fpr
youden_fpr = fpr[np.argmax(youden)]
youden_tpr = tpr[np.argmax(youden)]
cutoff = thresholds[np.argmax(youden)]  
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve')
plt.plot([youden_fpr, youden_fpr], [youden_fpr, youden_tpr],  color='black', lw=lw / 2, linestyle='--',
         label='Max Youden index (cutoff = {:.2f})'.format(cutoff))
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
y_final =  1 * (y_mean > cutoff)
evaluation(y_test, y_final)

In [ ]:
meta_train = y_prob_train.values; len(meta_train)

In [ ]:
# Neural network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

N_features = meta_train.shape[1]
BATCH_SIZE = 128

meta = Sequential()
meta.add(Dense(N_features, activation='relu', use_bias = True))
model.add(Dropout(0.5))
meta.add(Dense(1, activation='sigmoid', use_bias=True))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
meta.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['accuracy'])

# Fit the meta
meta.fit(meta_train, y_train, epochs=20, batch_size=BATCH_SIZE)

In [ ]:
meta_test = y_prob_test.values
meta_pred = meta.predict_classes(meta_test)
evaluation(y_test, meta_pred)

In [ ]:
non_stand_sim_test = similarity(G_master, test_pairs); non_stand_sim_test.head()

In [ ]:
sim_test = scaler.transform(non_stand_sim_test); sim_test

In [ ]:
final_test = pd.DataFrame(
    {'lr': [x[1] for x in clf_lr.predict_proba(sim_test)],
     'rf': [x[1] for x in clf_rf.predict_proba(sim_test)],
     'svm': [x[1] for x in clf_svm.predict_proba(sim_test)],
     'ada': [x[1] for x in clf_ada.predict_proba(sim_test)],
     'nn': [x[0] for x in clf_nn.predict(sim_test)]
})
final_test.head(n = 10)

In [ ]:
final_mean = final_test.apply(np.mean, axis = 1)
final_pred = 1 * (final_mean > cutoff); final_pred

In [ ]:
row_list = []
for i in range(2000):
    pair = test_pairs[i]
    row = {}
    row['Id'] = i + 1
    row['Predictions'] = final_pred[i]
    row_list.append(row)

predictions = pd.DataFrame(row_list)
predictions.to_csv('predictions.csv', index=False)